In [1]:
import os
import pandas as pd
import db_mysql as db
import re

fasce = ['2018-2017', '2016-2015', '2014-2013', '2012-2011', '2010-2009']
categorie = ['PANDA', 'GIRAFFA', 'ELEFANTE', 'ZEBRE', 'SCIMMIA', 'TIGRE MASCHILE', 'TIGRE FEMMINILE', 'LEONE MASCHILE', 'LEONE FEMMINILE']
nomi_kata = {
    ('PANDA', 1): 'TAIKYOKU SHODAN',
    ('PANDA', 2): 'TAIKYOKU SHODAN',
    ('PANDA', 3): 'TAIKYOKU SHODAN',
    ('GIRAFFA', 1): 'TAIKYOKU SHODAN',
    ('GIRAFFA', 2): 'TAIKYOKU SHODAN',
    ('GIRAFFA', 3): 'HEIAN SHODAN',
    ('ELEFANTE', 1): 'HEIAN SHODAN',
    ('ELEFANTE', 2): 'HEIAN SHODAN',
    ('ELEFANTE', 3): 'HEIAN NIDAN', 
    ('ZEBRE', 1): 'HEIAN SHODAN',
    ('ZEBRE', 2): 'HEIAN NIDAN',
    ('ZEBRE', 3): 'HEIAN SANDAN', 
    ('SCIMMIA', 1): 'HEIAN NIDAN',
    ('SCIMMIA', 2): 'HEIAN SANDAN',
    ('SCIMMIA', 3): 'HEIAN YONDAN', 
    ('TIGRE MASCHILE', 1): 'HEIAN SANDAN',
    ('TIGRE MASCHILE', 2): 'HEIAN YONDAN',
    ('TIGRE MASCHILE', 3): 'HEIAN GODAN', 
    ('TIGRE FEMMINILE', 1): 'HEIAN SANDAN',
    ('TIGRE FEMMINILE', 2): 'HEIAN YONDAN',
    ('TIGRE FEMMINILE', 3): 'HEIAN GODAN', 
    ('LEONE MASCHILE', 1): 'HEIAN GODAN',
    ('LEONE MASCHILE', 2): 'TEKKI SHODAN',
    ('LEONE MASCHILE', 3): 'BASSAI DAI', 
    ('LEONE FEMMINILE', 1): 'HEIAN GODAN',
    ('LEONE FEMMINILE', 2): 'TEKKI SHODAN',
    ('LEONE FEMMINILE', 3): 'BASSAI DAI'
}

def carica_singoli_da_cartelle(base_dir):
    """
    Carica i file xlsx presenti nelle sottocartelle di una cartella base in un DataFrame.
    
    Argomenti:
    base_dir (str): Percorso della cartella base.
    
    Ritorno:
    dict: Un dizionario in cui le chiavi sono i nomi delle cartelle e i valori sono i DataFrame corrispondenti ai file xlsx.
    """
    dati = {}
    for cartella in os.listdir(base_dir):
        cartella_path = os.path.join(base_dir, cartella)
        if os.path.isdir(cartella_path):
            dati_cartella = {}
            for file in os.listdir(cartella_path):
                file_path = os.path.join(cartella_path, file)
                if file.endswith('.xlsx'):
                    df = pd.read_excel(file_path, sheet_name='Singoli', header=6)
                    dati_cartella[file] = df
            dati[cartella] = dati_cartella
    return dati

# utilizzo:
cartella_base = 'Iscrizioni'
dati = carica_singoli_da_cartelle(cartella_base)

In [2]:
df_singoli = pd.DataFrame()
for cartella, dati_cartella in dati.items():
    for file, df in dati_cartella.items():
        df.insert(1, 'SOCIETA', cartella)
        df_singoli = pd.concat([df_singoli, df], axis=0)

df_singoli = df_singoli.rename(columns={'ANNO NASCITA [2009-2018]': 'ANNO NASCITA'})
df_singoli = df_singoli.drop(columns=['Unnamed: 0', 'QUOTA DI ISCRIZIONE', 'CONVALIDA FINALE'])
df_singoli.index = df_singoli.index + 1
df_singoli.index.name = 'N_ISCRITTO_SOCIETA'
df_singoli = df_singoli.reset_index()
df_singoli.index = df_singoli.index + 1
df_singoli.index.name = 'ID_ISCRITTO'

# Converte tutto in maiuscolo
df_singoli = df_singoli.map(lambda x: x.upper() if isinstance(x, str) else x)

# Suddivide le marroni e nere in categorie distinte in base al sesso
df_singoli.loc[(df_singoli['CATEGORIA'] == 'TIGRE') & (df_singoli['SESSO'] == 'MASCHIO'), 'CATEGORIA'] = 'TIGRE MASCHILE'
df_singoli.loc[(df_singoli['CATEGORIA'] == 'TIGRE') & (df_singoli['SESSO'] == 'FEMMINA'), 'CATEGORIA'] = 'TIGRE FEMMINILE'
df_singoli.loc[(df_singoli['CATEGORIA'] == 'LEONE') & (df_singoli['SESSO'] == 'MASCHIO'), 'CATEGORIA'] = 'LEONE MASCHILE'
df_singoli.loc[(df_singoli['CATEGORIA'] == 'LEONE') & (df_singoli['SESSO'] == 'FEMMINA'), 'CATEGORIA'] = 'LEONE FEMMINILE'

# Crea la colonna per fascia d'età
df_singoli['FASCIA'] = ''
df_singoli.loc[(df_singoli['ANNO NASCITA'] == 2018) | (df_singoli['ANNO NASCITA'] == 2017), 'FASCIA'] = '2018-2017'
df_singoli.loc[(df_singoli['ANNO NASCITA'] == 2016) | (df_singoli['ANNO NASCITA'] == 2015), 'FASCIA'] = '2016-2015'
df_singoli.loc[(df_singoli['ANNO NASCITA'] == 2014) | (df_singoli['ANNO NASCITA'] == 2013), 'FASCIA'] = '2014-2013'
df_singoli.loc[(df_singoli['ANNO NASCITA'] == 2012) | (df_singoli['ANNO NASCITA'] == 2011), 'FASCIA'] = '2012-2011'
df_singoli.loc[(df_singoli['ANNO NASCITA'] == 2010) | (df_singoli['ANNO NASCITA'] == 2009), 'FASCIA'] = '2010-2009'

In [3]:
df_singoli.to_parquet('Importati/singoli.parquet')
df_singoli

,N_ISCRITTO_SOCIETA,SOCIETA,COGNOME,NOME,SESSO,ANNO NASCITA,CINTURA,CATEGORIA,FASCIA
ID_ISCRITTO,,,,,,,,,
1,1,DOJO SHOTOKAN TREVISO,PESANTE,GIULIA,FEMMINA,2015,GIALLA/ARANCIO,ELEFANTE,2016-2015
2,2,DOJO SHOTOKAN TREVISO,NASINI,ANTONIO,MASCHIO,2014,GIALLA/ARANCIO,ELEFANTE,2014-2013
3,3,DOJO SHOTOKAN TREVISO,VETTOR,PIETRO,MASCHIO,2017,GIALLA,GIRAFFA,2018-2017
4,4,DOJO SHOTOKAN TREVISO,D'IORIO,FRANCESCO,MASCHIO,2016,ARANCIO/VERDE,ZEBRE,2016-2015
5,5,DOJO SHOTOKAN TREVISO,VASSOTTO,ALICE,FEMMINA,2016,GIALLA/ARANCIO,ELEFANTE,2016-2015
...,...,...,...,...,...,...,...,...,...
264,12,TSKS,BALDOVIN,PIETRO,MASCHIO,2015,GIALLA,GIRAFFA,2016-2015
265,13,TSKS,ESEGIO,CARLOTTA,FEMMINA,2010,BLU/MARRONE,TIGRE FEMMINILE,2010-2009
266,14,TSKS,FIORI,EMMA,FEMMINA,2010,MARRONE,TIGRE FEMMINILE,2010-2009


In [4]:
import os
import pandas as pd

def carica_squadre_da_cartelle(base_dir):
    """
    Carica i file xlsx presenti nelle sottocartelle di una cartella base in un DataFrame.
    
    Argomenti:
    base_dir (str): Percorso della cartella base.
    
    Ritorno:
    dict: Un dizionario in cui le chiavi sono i nomi delle cartelle e i valori sono i DataFrame corrispondenti ai file xlsx.
    """
    dati = {}
    for cartella in os.listdir(base_dir):
        cartella_path = os.path.join(base_dir, cartella)
        if os.path.isdir(cartella_path):
            dati_cartella = {}
            for file in os.listdir(cartella_path):
                file_path = os.path.join(cartella_path, file)
                if file.endswith('.xlsx'):
                    df = pd.read_excel(file_path, sheet_name='Squadre', header=4)
                    dati_cartella[file] = df
            dati[cartella] = dati_cartella
    return dati

# utilizzo:
cartella_base = 'Iscrizioni'
dati = carica_squadre_da_cartelle(cartella_base)

In [5]:
df_tot_squadra_A = pd.DataFrame()
df_tot_squadra_B = pd.DataFrame()
for cartella, dati_cartella in dati.items():
    for file, df in dati_cartella.items():
        df.insert(1, 'SOCIETA', cartella)
        squadra1 = df.iloc[0:3]
        squadra2 = df.iloc[9:12]
        df_tot_squadra_A = pd.concat([df_tot_squadra_A, squadra1], axis=0)
        df_tot_squadra_B = pd.concat([df_tot_squadra_B, squadra2], axis=0)

df_tot_squadra_A = df_tot_squadra_A.rename(columns={'ANNO NASCITA [2009-2018]': 'ANNO NASCITA'})
df_tot_squadra_A = df_tot_squadra_A.drop(columns=['Unnamed: 0', 'CONVALIDA'])
df_tot_squadra_A.index = df_tot_squadra_A.index + 1
df_tot_squadra_A.index.name = 'N_ISCRITTO_SOCIETA'
df_tot_squadra_A = df_tot_squadra_A.dropna(subset=['COGNOME'])
df_tot_squadra_A = df_tot_squadra_A.reset_index()
df_tot_squadra_A.index = df_tot_squadra_A.index + 1
df_tot_squadra_A.index.name = 'ID_ISCRITTO'

df_tot_squadra_B = df_tot_squadra_B.rename(columns={'ANNO NASCITA [2009-2018]': 'ANNO NASCITA'})
df_tot_squadra_B = df_tot_squadra_B.drop(columns=['Unnamed: 0', 'CONVALIDA'])
df_tot_squadra_B.index = df_tot_squadra_B.index + 1
df_tot_squadra_B.index.name = 'N_ISCRITTO_SOCIETA'
df_tot_squadra_B = df_tot_squadra_B.dropna(subset=['COGNOME'])
df_tot_squadra_B = df_tot_squadra_B.reset_index()
df_tot_squadra_B.index = df_tot_squadra_B.index + 1
df_tot_squadra_B.index.name = 'ID_ISCRITTO'

In [6]:
df_tot_squadra_A.to_parquet('Importati/squadra_A.parquet')
df_tot_squadra_A

,N_ISCRITTO_SOCIETA,SOCIETA,COGNOME,NOME,SESSO,ANNO NASCITA,CINTURA,CATEGORIA
ID_ISCRITTO,,,,,,,,
1,1,Karate Trevignano e Montebelluna,TANG,YIZI,Femmina,2009,MARRONE,TIGRE
2,2,Karate Trevignano e Montebelluna,CAVALLIN,ANITA,Femmina,2009,MARRONE,TIGRE
3,3,Karate Trevignano e Montebelluna,SEMENZATO,KIARA,Femmina,2011,MARRONE,TIGRE
4,1,Ki Kai Dojo Carbonera,SCALDAFERRI,JACOPO,Maschio,2013,BLU,SCIMMIA
5,2,Ki Kai Dojo Carbonera,FAVA,SAMUELE,Maschio,2013,VERDE/BLU,SCIMMIA
6,3,Ki Kai Dojo Carbonera,PIERAGNOLO,LEONE,Maschio,2015,VERDE/BLU,SCIMMIA
7,1,Scuola Karate Badoere,VEDELAGO,EVA,Femmina,2009,NERA,LEONE
8,2,Scuola Karate Badoere,FANTON,EMMA,Femmina,2009,NERA,LEONE
9,3,Scuola Karate Badoere,CARRARO,GIORGIA,Femmina,2009,NERA,LEONE


In [7]:
df_tot_squadra_B.to_parquet('Importati/squadra_B.parquet')
df_tot_squadra_B

,N_ISCRITTO_SOCIETA,SOCIETA,COGNOME,NOME,SESSO,ANNO NASCITA,CINTURA,CATEGORIA
ID_ISCRITTO,,,,,,,,
1,10,Karate Sacile,LAMBARKI,MALAC,Femmina,2014,GIALLA/ARANCIO,ELEFANTE
2,11,Karate Sacile,DONVITO,FRANCESCA,Femmina,2015,GIALLA/ARANCIO,ELEFANTE
3,12,Karate Sacile,GHEORGHITA,NATALIE,Femmina,2015,GIALLA/ARANCIO,ELEFANTE
4,10,Karate Trevignano e Montebelluna,MARCON,SOFIA,Femmina,2015,GIALLA/ARANCIO,ELEFANTE
5,11,Karate Trevignano e Montebelluna,VOLPATO,NICOLE,Femmina,2016,GIALLA/ARANCIO,ELEFANTE
6,12,Karate Trevignano e Montebelluna,GASPARETTO,REBECCA,Femmina,2014,GIALLA/ARANCIO,ELEFANTE
7,10,Ki Kai Dojo Carbonera,AGRIMI,GIACOMO,Maschio,2010,GIALLA/ARANCIO,ELEFANTE
8,11,Ki Kai Dojo Carbonera,MEZZAVILLA,NICOLO',Maschio,2010,GIALLA/ARANCIO,ELEFANTE
9,12,Ki Kai Dojo Carbonera,ROCCO,FEDERICO,Maschio,2010,ARANCIO/VERDE,ZEBRE


## Salva società su db

In [8]:
società = df_singoli['SOCIETA'].str.upper().drop_duplicates().to_frame().rename(columns={'SOCIETA': 'id_societa'})
temp_df1 = df_tot_squadra_A['SOCIETA'].str.upper().drop_duplicates().to_frame().rename(columns={'SOCIETA': 'id_societa'})
temp_df2 = df_tot_squadra_B['SOCIETA'].str.upper().drop_duplicates().to_frame().rename(columns={'SOCIETA': 'id_societa'})
società = pd.concat([società, temp_df1, temp_df2], axis=0)
società['id_societa'] = società['id_societa'].str.upper()
società['nome'] = società['id_societa']
società['id_societa'] = società['id_societa'].apply(lambda x: re.sub(r'\s', '', x))
società.drop_duplicates(subset='id_societa', inplace=True)
società.set_index('id_societa', inplace=True)
società['n_ordine'] = range(1, len(società) + 1)

def salva_società(df):
    db.execute_query('DELETE FROM societa', None)
    for index, row in df.iterrows():
        # Aggiunta degli indici alla lista dei valori
        values_with_indices = [index] + [None if (pd.isna(value)) else value for value in row]
        
        # Aggiunta degli indici alla lista dei nomi delle colonne
        columns_with_indices = ['id_societa'] + df.columns.tolist()
        
        # Creazione della query di inserimento con i parametri
        sql = f"INSERT INTO societa ({', '.join(columns_with_indices)}) VALUES ({', '.join(['%s' for _ in range(len(values_with_indices))])})"
        
        # Esecuzione della query utilizzando la funzione execute_query
        db.execute_query(sql, tuple(values_with_indices))
        
salva_società(società)
società

,nome,n_ordine
id_societa,,
DOJOSHOTOKANTREVISO,DOJO SHOTOKAN TREVISO,1
KARATESACILE,KARATE SACILE,2
KARATETREVIGNANOEMONTEBELLUNA,KARATE TREVIGNANO E MONTEBELLUNA,3
KIKAIDOJOCARBONERA,KI KAI DOJO CARBONERA,4
KORIYAMAASD,KORIYAMA ASD,5
RENBUKANMOGLIANO,REN BU KAN MOGLIANO,6
RENBUKANSPRESIANO,REN BU KAN SPRESIANO,7
RENBUKANTREVISO,REN BU KAN TREVISO,8
SCUOLAKARATEBADOERE,SCUOLA KARATE BADOERE,9


## Salva atleti su db

In [13]:
atleti = df_singoli.drop(columns=['N_ISCRITTO_SOCIETA'])
atleti.rename(inplace=True, columns={'COGNOME': 'cognome', 'NOME': 'nome', 'SESSO': 'sesso', 'ANNO NASCITA': 'anno_nascita', 'CINTURA': 'cintura', 'SOCIETA': 'societa', 'CATEGORIA': 'categoria', 'FASCIA': 'fascia'})
atleti['sesso'] = atleti['sesso'].apply(lambda x: x[0].upper())
atleti['cognome'] = atleti['cognome'].apply(lambda x: x.upper())
atleti['nome'] = atleti['nome'].apply(lambda x: x.upper())
atleti['cintura'] = atleti['cintura'].apply(lambda x: x.upper())
atleti['societa'] = atleti['societa'].apply(lambda x: x.upper())
atleti['categoria'] = atleti['categoria'].apply(lambda x: x.upper())
atleti['id_atleta'] = atleti.apply(lambda row: str(row['cognome']) + '_' + str(row['nome']) + '_' + str(row['sesso']) + '_' + str(row['anno_nascita']), axis=1)
atleti['id_atleta'] = atleti['id_atleta'].apply(lambda x: re.sub(r'\s', '', x))

iscrizioni_individuali = pd.DataFrame()
iscrizioni_individuali['id_societa'] = atleti['societa'].apply(lambda x: re.sub(r'\s', '', x))
iscrizioni_individuali['id_atleta'] = atleti['id_atleta']
iscrizioni_individuali['fascia_iscrizione'] = atleti['fascia']
iscrizioni_individuali['categoria_iscrizione'] = atleti['categoria']
iscrizioni_individuali['id_iscrizione'] = range(1, len(iscrizioni_individuali) + 1)
iscrizioni_individuali.set_index('id_iscrizione', inplace=True)

def salva_iscrizioni_individuali(df):
    db.execute_query('DELETE FROM iscrizioni_individuali', None)
    for index, row in df.iterrows():
        # Aggiunta degli indici alla lista dei valori
        values_with_indices = [index] + [None if (pd.isna(value)) else value for value in row]
        
        # Aggiunta degli indici alla lista dei nomi delle colonne
        columns_with_indices = ['id_iscrizione'] + df.columns.tolist()
        
        # Creazione della query di inserimento con i parametri
        sql = f"INSERT INTO iscrizioni_individuali ({', '.join(columns_with_indices)}) VALUES ({', '.join(['%s' for _ in range(len(values_with_indices))])})"
        
        # Esecuzione della query utilizzando la funzione execute_query
        db.execute_query(sql, tuple(values_with_indices))

salva_iscrizioni_individuali(iscrizioni_individuali)

atleti = atleti.drop(columns=['societa', 'categoria', 'fascia'])

temp_df1 = df_tot_squadra_A.drop(columns=['N_ISCRITTO_SOCIETA'])
temp_df1.rename(inplace=True, columns={'COGNOME': 'cognome', 'NOME': 'nome', 'SESSO': 'sesso', 'ANNO NASCITA': 'anno_nascita', 'CINTURA': 'cintura', 'SOCIETA': 'societa', 'CATEGORIA': 'categoria'})
temp_df1['sesso'] = temp_df1['sesso'].apply(lambda x: x[0].upper())
temp_df1['cognome'] = temp_df1['cognome'].apply(lambda x: x.upper())
temp_df1['nome'] = temp_df1['nome'].apply(lambda x: x.upper())
temp_df1['cintura'] = temp_df1['cintura'].apply(lambda x: x.upper())
temp_df1['societa'] = temp_df1['societa'].apply(lambda x: x.upper())
temp_df1['categoria'] = temp_df1['categoria'].apply(lambda x: x.upper())
temp_df1['id_atleta'] = temp_df1.apply(lambda row: str(row['cognome']) + '_' + str(row['nome']) + '_' + str(row['sesso']) + '_' + str(row['anno_nascita']), axis=1)
temp_df1['id_atleta'] = temp_df1['id_atleta'].apply(lambda x: re.sub(r'\s', '', x))
temp_df1['squadra'] = 'SQUADRA_A'

temp_df2 = df_tot_squadra_B.drop(columns=['N_ISCRITTO_SOCIETA'])
temp_df2.rename(inplace=True, columns={'COGNOME': 'cognome', 'NOME': 'nome', 'SESSO': 'sesso', 'ANNO NASCITA': 'anno_nascita', 'CINTURA': 'cintura', 'SOCIETA': 'societa', 'CATEGORIA': 'categoria'})
temp_df2['sesso'] = temp_df2['sesso'].apply(lambda x: x[0].upper())
temp_df2['cognome'] = temp_df2['cognome'].apply(lambda x: x.upper())
temp_df2['nome'] = temp_df2['nome'].apply(lambda x: x.upper())
temp_df2['cintura'] = temp_df2['cintura'].apply(lambda x: x.upper())
temp_df2['societa'] = temp_df2['societa'].apply(lambda x: x.upper())
temp_df2['categoria'] = temp_df2['categoria'].apply(lambda x: x.upper())
temp_df2['id_atleta'] = temp_df2.apply(lambda row: str(row['cognome']) + '_' + str(row['nome']) + '_' + str(row['sesso']) + '_' + str(row['anno_nascita']), axis=1)
temp_df2['id_atleta'] = temp_df2['id_atleta'].apply(lambda x: re.sub(r'\s', '', x))
temp_df2['squadra'] = 'SQUADRA_B'

iscrizioni_squadre_temp = pd.concat([temp_df1, temp_df2], axis=0)
iscrizioni_squadre_temp['societa'] = iscrizioni_squadre_temp['societa'].apply(lambda x: re.sub(r'\s', '', x))
iscrizioni_squadre_temp['id_squadra'] = iscrizioni_squadre_temp['societa'] + '_' + iscrizioni_squadre_temp['squadra']
iscrizioni_squadre = iscrizioni_squadre_temp[['id_squadra', 'id_atleta']]
iscrizioni_squadre['id_iscrizione'] = range(1, len(iscrizioni_squadre) + 1)
iscrizioni_squadre.set_index('id_iscrizione', inplace=True)

def salva_iscrizioni_squadre(df):
    db.execute_query('DELETE FROM iscrizioni_squadre', None)
    for index, row in df.iterrows():
        # Aggiunta degli indici alla lista dei valori
        values_with_indices = [index] + [None if (pd.isna(value)) else value for value in row]
        
        # Aggiunta degli indici alla lista dei nomi delle colonne
        columns_with_indices = ['id_iscrizione'] + df.columns.tolist()
        
        # Creazione della query di inserimento con i parametri
        sql = f"INSERT INTO iscrizioni_squadre ({', '.join(columns_with_indices)}) VALUES ({', '.join(['%s' for _ in range(len(values_with_indices))])})"
        
        # Esecuzione della query utilizzando la funzione execute_query
        db.execute_query(sql, tuple(values_with_indices))

salva_iscrizioni_squadre(iscrizioni_squadre)

squadre = iscrizioni_squadre_temp[['id_squadra', 'societa', 'squadra']]
squadre.rename(inplace=True, columns={'societa': 'id_societa', 'squadra': 'nome_squadra'})
squadre.drop_duplicates(subset='id_squadra', inplace=True)
squadre.set_index('id_squadra', inplace=True)
def salva_squadre(df):
    db.execute_query('DELETE FROM squadre', None)
    for index, row in df.iterrows():
        # Aggiunta degli indici alla lista dei valori
        values_with_indices = [index] + [None if (pd.isna(value)) else value for value in row]
        
        # Aggiunta degli indici alla lista dei nomi delle colonne
        columns_with_indices = ['id_squadra'] + df.columns.tolist()
        
        # Creazione della query di inserimento con i parametri
        sql = f"INSERT INTO squadre ({', '.join(columns_with_indices)}) VALUES ({', '.join(['%s' for _ in range(len(values_with_indices))])})"
        
        # Esecuzione della query utilizzando la funzione execute_query
        db.execute_query(sql, tuple(values_with_indices))

salva_squadre(squadre)

temp_df1 = temp_df1.drop(columns=['societa', 'categoria', 'squadra'])
temp_df2 = temp_df2.drop(columns=['societa', 'categoria', 'squadra'])
atleti = pd.concat([atleti, temp_df1, temp_df2], axis=0)

atleti.drop_duplicates(subset='id_atleta', inplace=True)
atleti.set_index('id_atleta', inplace=True)
atleti = atleti.sort_index()
atleti['n_ordine'] = range(1, len(atleti) + 1)

def salva_atleti(df):
    db.execute_query('DELETE FROM atleti', None)
    for index, row in df.iterrows():
        # Aggiunta degli indici alla lista dei valori
        values_with_indices = [index] + [None if (pd.isna(value)) else value for value in row]
        
        # Aggiunta degli indici alla lista dei nomi delle colonne
        columns_with_indices = ['id_atleta'] + df.columns.tolist()
        
        # Creazione della query di inserimento con i parametri
        sql = f"INSERT INTO atleti ({', '.join(columns_with_indices)}) VALUES ({', '.join(['%s' for _ in range(len(values_with_indices))])})"
        
        # Esecuzione della query utilizzando la funzione execute_query
        db.execute_query(sql, tuple(values_with_indices))
        
salva_atleti(atleti)


C:\Users\enrico\AppData\Local\Temp\ipykernel_7176\1780104089.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iscrizioni_squadre['id_iscrizione'] = range(1, len(iscrizioni_squadre) + 1)
C:\Users\enrico\AppData\Local\Temp\ipykernel_7176\1780104089.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  squadre.rename(inplace=True, columns={'societa': 'id_societa', 'squadra': 'nome_squadra'})
C:\Users\enrico\AppData\Local\Temp\ipykernel_7176\1780104089.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveat